In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
from matplotlib.ticker import MaxNLocator

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss, make_scorer, recall_score, accuracy_score, precision_score, f1_score, cohen_kappa_score,roc_curve, auc

import os, eli5
from eli5.sklearn import PermutationImportance
from pdpbox import pdp
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn import svm
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity 


Using TensorFlow backend.


Get all previous years data form 2003 onward. Using only teams that have made the tourney. 298 of approx. 360 teams have made the tourney in the past 10 years.

In [2]:
df_tourney_all = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/NCAATourneyCompactResults.csv')
df = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/RegularSeasonDetailedResults.csv')
df_team_spellings = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/TeamSpellings.csv',encoding = "ISO-8859-1")
df_seeds = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/NCAATourneySeeds.csv')

#Limit to only tourney teams
past_trny_teams = list(set(list(df_tourney_all.WTeamID.unique()) + list(df_tourney_all.LTeamID.unique()))) 
len(past_trny_teams)

300

Get 2019 tourney teams. Correctly there are 68 teams. Format of submission is an id of 'year'_'team1id'_'team2id' and a predicted probability of team 1 winning. 

To get unique teams split by _ to get both ids, create two sets of lists combine and get unique ids.

In [3]:
df_predict = pd.read_csv('mens-machine-learning-competition-2019/input/SampleSubmissionStage2.csv')
grp1 = list(set(df_predict.apply(lambda row: row.ID.split('_')[1], axis=1)))
grp2 = list(set(df_predict.apply(lambda row: row.ID.split('_')[2], axis=1)))
crnt_trny_teams = list(set(grp1 + grp2))
len(crnt_trny_teams)

68

Include more descriptive data on the teams involved

In [4]:
#Team name with id, first and last D1 season
df_teams = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/Teams.csv')
#Team name matched with conf abbrev and for each season
df_team_conferences = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/TeamConferences.csv')
#conference abbrev with full name
df_conferences = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/Conferences.csv')

df_conference_names = df_team_conferences.merge(df_conferences, on=['ConfAbbrev'])

#Create tables to use as reference for combining real names with ids for full game results
win_teams = df_teams.rename(columns={'TeamID':'WTeamID'})[['WTeamID', 'TeamName']]
win_confs = df_conference_names.rename(columns={'TeamID':'WTeamID'})[['Season', 'WTeamID', 'Description']]
lose_teams = df_teams.rename(columns={'TeamID':'LTeamID'})[['LTeamID', 'TeamName']]
lose_confs = df_conference_names.rename(columns={'TeamID':'LTeamID'})[['Season', 'LTeamID', 'Description']]

# Merge winning team name and conference, losing team name and conference with season results:
df = df.merge(win_teams, on='WTeamID').rename(columns={'TeamName': 'WTeamName'}) \
.merge(win_confs, on=['Season', 'WTeamID']).rename(columns={'Description': 'WConfName'}) \
.merge(lose_teams, on='LTeamID').rename(columns={'TeamName': 'LTeamName'}) \
.merge(lose_confs, on=['Season', 'LTeamID']).rename(columns={'Description': 'LConfName'})
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WTeamName,WConfName,LTeamName,LConfName
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,Alabama,Southeastern Conference,Oklahoma,Big 12 Conference
1,2003,98,1400,67,1328,61,H,0,25,64,5,14,12,20,18,24,10,10,5,4,16,22,52,7,14,10,15,9,25,5,14,5,3,20,Texas,Big 12 Conference,Oklahoma,Big 12 Conference
2,2003,124,1400,76,1328,71,A,0,27,50,6,11,16,25,9,28,13,13,5,0,21,25,60,8,21,13,23,15,19,16,9,8,6,23,Texas,Big 12 Conference,Oklahoma,Big 12 Conference
3,2003,114,1281,67,1328,52,H,0,24,55,7,15,12,14,10,24,14,5,4,5,18,15,55,5,23,17,19,17,22,8,11,1,3,17,Missouri,Big 12 Conference,Oklahoma,Big 12 Conference
4,2003,70,1329,48,1328,46,H,0,19,49,1,8,9,16,10,28,4,8,4,6,17,18,55,3,12,7,15,15,23,10,6,4,3,18,Oklahoma St,Big 12 Conference,Oklahoma,Big 12 Conference


In [5]:
#Add 2pt fg data
df['WFGM2'] = df.WFGM - df.WFGM3
df['WFGA2'] = df.WFGA - df.WFGA3
df['LFGM2'] = df.LFGM - df.LFGM3
df['LFGA2'] = df.LFGA - df.LFGA3

In [6]:
print('These are the {} conferences that have participated in NCAA Division I men’s basketball since 2003, including the number of wins in the dataframe for each:'.format(len(df.WConfName.value_counts())))
df.WConfName.value_counts()

These are the 36 conferences that have participated in NCAA Division I men’s basketball since 2003, including the number of wins in the dataframe for each:


Big East Conference                   4310
Atlantic Coast Conference             4175
Southeastern Conference               4081
Big Ten Conference                    3893
Atlantic 10 Conference                3882
Big 12 Conference                     3608
Conference USA                        3514
Mid-American Conference               3141
Colonial Athletic Association         2823
Sun Belt Conference                   2811
Missouri Valley Conference            2807
Mountain West Conference              2727
Metro Atlantic Athletic Conference    2624
Southern Conference                   2544
Ohio Valley Conference                2460
Northeast Conference                  2450
Southland Conference                  2433
Horizon League                        2374
Mid-Eastern Athletic Conference       2352
West Coast Conference                 2343
Western Athletic Conference           2284
Atlantic Sun Conference               2216
Big Sky Conference                    2119
America Eas

In [7]:
print('Season  #Games:')
df.Season.value_counts()

Season  #Games:


2019    5463
2018    5405
2017    5395
2016    5369
2014    5362
2015    5354
2013    5320
2010    5263
2012    5253
2009    5249
2011    5246
2008    5163
2007    5043
2006    4757
2005    4675
2003    4616
2004    4571
Name: Season, dtype: int64

Winning team description of stats

In [8]:
df[['WFGA', 'WFTA', 'WTO', 'WOR', 'WScore', 'WFGM2', 'WFGM3', 'WFGM', 'WFTM', 'WDR', 'WAst', 'WStl', 'WBlk', 'WPF']].describe()

,WFGA,WFTA,WTO,WOR,WScore,WFGM2,WFGM3,WFGM,WFTM,WDR,WAst,WStl,WBlk,WPF
count,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000
mean,55.174266,22.659113,12.975464,10.900290,75.429580,18.996846,7.114646,26.111492,16.091950,25.677066,14.738423,7.008777,3.830899,17.451477
std,7.564707,8.123213,4.026165,4.132544,11.131958,4.691893,3.070932,4.700220,6.249622,4.909033,4.412361,3.123401,2.460668,4.136628
min,27.000000,0.000000,1.000000,0.000000,34.000000,2.000000,0.000000,10.000000,0.000000,5.000000,1.000000,0.000000,0.000000,3.000000
25%,50.000000,17.000000,10.000000,8.000000,68.000000,16.000000,5.000000,23.000000,12.000000,22.000000,12.000000,5.000000,2.000000,15.000000
50%,55.000000,22.000000,13.000000,11.000000,75.000000,19.000000,7.000000,26.000000,16.000000,26.000000,14.000000,7.000000,3.000000,17.000000
75%,60.000000,28.000000,16.000000,13.000000,82.000000,22.000000,9.000000,29.000000,20.000000,29.000000,18.000000,9.000000,5.000000,20.000000
max,103.000000,67.000000,33.000000,38.000000,144.000000,48.000000,26.000000,56.000000,48.000000,53.000000,40.000000,26.000000,21.000000,41.000000


In [9]:
df[['LFGA', 'LFTA', 'LTO', 'LOR', 'LScore', 'LFGM2', 'LFGM3', 'LFGM', 'LFTM', 'LDR', 'LAst', 'LStl', 'LBlk', 'LPF']].describe()

,LFGA,LFTA,LTO,LOR,LScore,LFGM2,LFGM3,LFGM,LFTM,LDR,LAst,LStl,LBlk,LPF
count,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000,87504.000000
mean,56.392691,18.039244,14.313826,11.115526,63.437866,16.538307,6.058043,22.596350,12.187123,21.631525,11.439706,6.008422,2.870749,19.766811
std,7.686611,7.143685,4.407597,4.207721,10.974796,4.307365,2.781245,4.388046,5.368036,4.543547,3.732508,2.766675,2.037376,4.528199
min,26.000000,0.000000,0.000000,0.000000,20.000000,2.000000,0.000000,6.000000,0.000000,4.000000,0.000000,0.000000,0.000000,4.000000
25%,51.000000,13.000000,11.000000,8.000000,56.000000,14.000000,4.000000,20.000000,8.000000,19.000000,9.000000,4.000000,1.000000,17.000000
50%,56.000000,18.000000,14.000000,11.000000,63.000000,16.000000,6.000000,22.000000,12.000000,21.000000,11.000000,6.000000,3.000000,20.000000
75%,61.000000,23.000000,17.000000,14.000000,70.000000,19.000000,8.000000,25.000000,16.000000,25.000000,14.000000,8.000000,4.000000,23.000000
max,106.000000,61.000000,41.000000,36.000000,140.000000,40.000000,22.000000,47.000000,42.000000,49.000000,31.000000,22.000000,18.000000,45.000000


# Create some new stats

In [10]:
# Winner stats related to offensive efficiency:
df['Wposs'] = df.apply(lambda row: row.WFGA + 0.475 * row.WFTA + row.WTO - row.WOR, axis=1)
df['Wshoot_eff'] = df.apply(lambda row: row.WScore / (row.WFGA + 0.475 * row.WFTA), axis=1)
df['Wscore_op'] = df.apply(lambda row: (row.WFGA + 0.475 * row.WFTA) / row.Wposs, axis=1)
df['Woff_rtg'] = df.apply(lambda row: row.WScore/row.Wposs*100, axis=1)

# Loser stats related to offensive efficiency:
df['Lposs'] = df.apply(lambda row: row.LFGA + 0.475 * row.LFTA + row.LTO - row.LOR, axis=1)
df['Lshoot_eff'] = df.apply(lambda row: row.LScore / (row.LFGA + 0.475 * row.LFTA), axis=1)
df['Lscore_op'] = df.apply(lambda row: (row.LFGA + 0.475 * row.LFTA) / row.Lposs, axis=1)
df['Loff_rtg'] = df.apply(lambda row: row.LScore/row.Lposs*100, axis=1)

# Defensive and net efficiency:
df['Wdef_rtg'] = df.apply(lambda row: row.Loff_rtg, axis=1)
df['Wsos'] = df.apply(lambda row: row.Woff_rtg - row.Loff_rtg, axis=1)
df['Ldef_rtg'] = df.apply(lambda row: row.Woff_rtg, axis=1)
df['Lsos'] = df.apply(lambda row: row.Loff_rtg - row.Woff_rtg, axis=1)

# Impact Estimate - 
# First calculate the teams' overall statistical contribution (the numerator):
Wie = df.apply(lambda row: row.WScore + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + (0.5 * row.WOR) + row.WAst + row.WStl + (0.5 * row.WBlk) - row.WPF - row.WTO, axis=1)
Lie = df.apply(lambda row: row.LScore + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + (0.5 * row.LOR) + row.LAst + row.LStl + (0.5 * row.LBlk) - row.LPF - row.LTO, axis=1)

# Then divide by the total game statistics (the denominator):
df['Wie'] = Wie / (Wie + Lie) * 100
df['Lie'] = Lie / (Lie + Wie) * 100

# Other winner stats:
df['Wts_pct'] = df.apply(lambda row: row.WScore / (2 * (row.WFGA + 0.475 * row.WFTA)) * 100, axis=1)
df['Wefg_pct'] = df.apply(lambda row: (row.WFGM2 + 1.5 * row.WFGM3) / row.WFGA, axis=1)
df['Worb_pct'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df['Wdrb_pct'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df['Wreb_pct'] = df.apply(lambda row: (row.Worb_pct + row.Wdrb_pct) / 2, axis=1)
df['Wto_poss'] = df.apply(lambda row: row.WTO / row.Wposs, axis=1)
df['Wft_rate'] = df.apply(lambda row: row.WFTM / row.WFGA, axis=1)
df['Wast_rtio'] = df.apply(lambda row: row.WAst / (row.WFGA + 0.475*row.WFTA + row.WTO + row.WAst) * 100, axis=1)
df['Wblk_pct'] = df.apply(lambda row: row.WBlk / row.LFGA2 * 100, axis=1)
df['Wstl_pct'] = df.apply(lambda row: row.WStl / row.Lposs * 100, axis=1)
df['W3pta_pct'] = df.apply(lambda row: row.WFGA3 / row.WFGA * 100, axis=1)

# Other loser stats:
df['Lts_pct'] = df.apply(lambda row: row.LScore / (2 * (row.LFGA + 0.475 * row.LFTA)) * 100, axis=1)
df['Lefg_pct'] = df.apply(lambda row: (row.LFGM2 + 1.5 * row.LFGM3) / row.LFGA, axis=1)
df['Lorb_pct'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
df['Ldrb_pct'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)
df['Lreb_pct'] = df.apply(lambda row: (row.Lorb_pct + row.Ldrb_pct) / 2, axis=1)
df['Lto_poss'] = df.apply(lambda row: row.LTO / row.Lposs, axis=1)
df['Lft_rate'] = df.apply(lambda row: row.LFTM / row.LFGA, axis=1)
df['Last_rtio'] = df.apply(lambda row: row.LAst / (row.LFGA + 0.475*row.LFTA + row.LTO + row.LAst) * 100, axis=1)
df['Lblk_pct'] = df.apply(lambda row: row.LBlk / row.WFGA2 * 100, axis=1)
df['Lstl_pct'] = df.apply(lambda row: row.LStl / row.Wposs * 100, axis=1)
df['L3pta_pct'] = df.apply(lambda row: row.LFGA3 / row.LFGA * 100, axis=1)

## Calculate Season Averages

In [11]:
# Initialize dataframe to hold season averages:
df_avgs = pd.DataFrame()
# Get and save number of wins and losses:
df_avgs['n_wins'] = df['WTeamID'].groupby([df.Season, df.WTeamID, df.WTeamName, df.WConfName]).count()
df_avgs['n_loss'] = df['LTeamID'].groupby([df.Season, df.LTeamID, df.LTeamName, df.LConfName]).count()

df_avgs['n_loss'].fillna(0, inplace=True)
# Calculate win percentages:
df_avgs['win_pct'] = df_avgs['n_wins'] / (df_avgs['n_wins'] + df_avgs['n_loss'])  
# df_avgs = df_avgs.reset_index()

In [12]:
def calc_avg(var):
    df_avgs['W' + var] = df['W' + var].groupby([df['Season'], df['WTeamID'], df.WTeamName, df.WConfName]).mean()
    df_avgs['L' + var] = df['L' + var].groupby([df['Season'], df['LTeamID'], df.LTeamName, df.LConfName]).mean()
    df_avgs[var] = df_avgs['W' + var] * df_avgs['win_pct'] + df_avgs['L' + var] * (1 - df_avgs['win_pct'])
    
    

In [13]:
# Calculate averages for games won:
stats = ['shoot_eff', 'score_op', 'off_rtg', 'def_rtg', 'sos', 'ts_pct', 'efg_pct', 'orb_pct','drb_pct', 'reb_pct',
        'to_poss', 'ft_rate', 'ie', 'ast_rtio', 'blk_pct', 'stl_pct', '3pta_pct']
for stat in stats:
    calc_avg(stat)

In [14]:
df_avgs.reset_index(inplace = True)
df_avgs[df_avgs['Season'] == 2014]

,Season,WTeamID,WTeamName,WConfName,n_wins,n_loss,win_pct,Wshoot_eff,Lshoot_eff,shoot_eff,Wscore_op,Lscore_op,score_op,Woff_rtg,Loff_rtg,off_rtg,Wdef_rtg,Ldef_rtg,def_rtg,Wsos,Lsos,sos,Wts_pct,Lts_pct,ts_pct,Wefg_pct,Lefg_pct,efg_pct,Worb_pct,Lorb_pct,orb_pct,Wdrb_pct,Ldrb_pct,drb_pct,Wreb_pct,Lreb_pct,reb_pct,Wto_poss,Lto_poss,to_poss,Wft_rate,Lft_rate,ft_rate,Wie,Lie,ie,Wast_rtio,Last_rtio,ast_rtio,Wblk_pct,Lblk_pct,blk_pct,Wstl_pct,Lstl_pct,stl_pct,W3pta_pct,L3pta_pct,3pta_pct
3723,2014,1101,Abilene Chr,Southland Conference,2,19.0,0.095238,1.267818,1.029811,1.052479,0.900292,0.892400,0.893152,114.118276,91.314594,93.486373,111.095779,117.242884,116.657445,3.022497,-25.928290,-23.171072,63.390886,51.490567,52.623930,0.594661,0.465271,0.477594,0.288123,0.242433,0.246784,0.673529,0.668514,0.668992,0.480826,0.455473,0.457888,0.206418,0.228608,0.226495,0.487993,0.307591,0.324772,55.662284,27.929458,30.570679,12.898381,11.434467,11.573888,5.141011,3.879733,3.999854,14.489609,7.936897,8.560965,23.434820,36.965690,35.677036
3724,2014,1102,Air Force,Mountain West Conference,10,18.0,0.357143,1.145266,1.020390,1.064989,0.931008,0.927116,0.928506,105.737066,94.153986,98.290800,95.174189,113.995464,107.273581,10.562876,-19.841478,-8.982780,57.263296,51.019512,53.249435,0.531742,0.483062,0.500448,0.300184,0.239385,0.261099,0.756765,0.668194,0.699826,0.528474,0.453790,0.480463,0.214855,0.198628,0.204423,0.350243,0.216457,0.264238,57.363544,37.085174,44.327449,15.248212,15.109399,15.158975,11.587385,8.349674,9.505999,9.025627,7.386298,7.971773,42.177427,41.424026,41.693098
3725,2014,1103,Akron,Mid-American Conference,21,12.0,0.636364,1.095640,0.954418,1.044287,0.985866,0.988131,0.986690,107.721860,94.072799,102.758565,91.637608,112.925200,99.378551,16.084252,-18.852401,3.380014,54.782023,47.720883,52.214336,0.527830,0.446478,0.498248,0.346313,0.353803,0.349037,0.695634,0.650426,0.679195,0.520974,0.502115,0.514116,0.190911,0.215649,0.199907,0.278052,0.222148,0.257723,63.059957,31.169947,51.463590,13.698896,10.233803,12.438862,11.853529,7.821124,10.387200,10.596377,8.629852,9.881277,37.347792,38.016004,37.590778
3726,2014,1104,Alabama,Southeastern Conference,12,19.0,0.387097,1.177969,0.998941,1.068242,0.966854,0.974246,0.971385,113.542736,96.788159,103.273802,92.334704,110.659198,103.565845,21.208033,-13.871039,-0.292044,58.898462,49.947037,53.412104,0.563151,0.454228,0.496392,0.324180,0.278303,0.296062,0.700841,0.635888,0.661031,0.512510,0.457096,0.478547,0.186134,0.179433,0.182027,0.312927,0.300351,0.305219,67.298433,37.315900,48.922042,14.467371,11.321883,12.539491,13.143789,10.586030,11.576130,12.925331,8.624841,10.289547,35.115879,32.889606,33.751389
3727,2014,1105,Alabama A&M,Southwest Athletic Conference,12,16.0,0.428571,1.107058,0.930731,1.006300,0.975963,0.932986,0.951405,107.651948,86.240077,95.416593,91.732585,106.163574,99.978865,15.919363,-19.923497,-4.562271,55.352922,46.536543,50.314991,0.521716,0.424503,0.466166,0.338843,0.232898,0.278303,0.682863,0.642710,0.659918,0.510853,0.437804,0.469111,0.196700,0.195934,0.196262,0.333862,0.246288,0.283820,68.357011,30.468211,46.706268,15.503872,12.707177,13.905761,12.420855,9.658582,10.842413,9.643422,8.474286,8.975344,33.828507,37.145219,35.723771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,2014,1460,Wright St,Horizon League,18,14.0,0.562500,1.156334,1.069280,1.118248,0.909536,0.902781,0.906580,104.451361,96.127189,100.809536,88.246031,107.119383,96.503122,16.205330,-10.992194,4.306413,57.816701,53.464008,55.912398,0.525232,0.503984,0.515936,0.250446,0.212184,0.233706,0.703003,0.693141,0.698688,0.476724,0.452663,0.466197,0.200925,0.202449,0.201591,0.410813,0.234047,0.333478,66.113549,41.455943,55.325846,15.113135,15.610618,15.330784,10.328134,7.190367,8.955361,10.174121,10.394371,10.270480,33

In [15]:
df_avgs = df_avgs.rename(columns={'WTeamID': 'TeamID', 'WTeamName': 'TeamName', 'WConfName': 'ConfName'})
df_avgs.head()

,Season,TeamID,TeamName,ConfName,n_wins,n_loss,win_pct,Wshoot_eff,Lshoot_eff,shoot_eff,Wscore_op,Lscore_op,score_op,Woff_rtg,Loff_rtg,off_rtg,Wdef_rtg,Ldef_rtg,def_rtg,Wsos,Lsos,sos,Wts_pct,Lts_pct,ts_pct,Wefg_pct,Lefg_pct,efg_pct,Worb_pct,Lorb_pct,orb_pct,Wdrb_pct,Ldrb_pct,drb_pct,Wreb_pct,Lreb_pct,reb_pct,Wto_poss,Lto_poss,to_poss,Wft_rate,Lft_rate,ft_rate,Wie,Lie,ie,Wast_rtio,Last_rtio,ast_rtio,Wblk_pct,Lblk_pct,blk_pct,Wstl_pct,Lstl_pct,stl_pct,W3pta_pct,L3pta_pct,3pta_pct
0,2003,1102,Air Force,Mountain West Conference,12,16.0,0.428571,1.385301,1.055690,1.196952,0.876488,0.867953,0.871611,120.972372,90.841475,103.754717,92.784626,112.088477,103.815398,28.187745,-21.247001,-0.060681,69.265074,52.784497,59.847601,0.692162,0.503591,0.584407,0.172188,0.165271,0.168235,0.636526,0.625778,0.630384,0.404357,0.395525,0.399310,0.191909,0.214990,0.205098,0.355877,0.243105,0.291436,75.526345,28.860022,48.859875,21.645310,14.672462,17.660825,8.457639,3.591274,5.676859,12.866850,9.388406,10.879167,53.690087,50.474281,51.852484
1,2003,1103,Akron,Mid-American Conference,13,14.0,0.481481,1.257318,1.062243,1.156168,0.955361,0.961669,0.958632,120.043926,101.763397,110.565133,106.613550,114.354975,110.627622,13.430376,-12.591578,-0.062489,62.865925,53.112160,57.808417,0.592604,0.484527,0.536564,0.318510,0.294003,0.305803,0.620109,0.633394,0.626998,0.469309,0.463699,0.466400,0.172876,0.185367,0.179353,0.401870,0.288288,0.342976,61.544613,41.154014,50.971710,17.655516,14.022883,15.771929,4.771096,7.523921,6.198487,9.896441,10.321618,10.116903,26.339581,31.260267,28.891047
2,2003,1104,Alabama,Southeastern Conference,17,11.0,0.607143,1.085554,0.949278,1.032017,1.006881,0.998515,1.003595,109.038086,94.610164,103.369974,89.448746,111.355747,98.055068,19.589340,-16.745583,5.314906,54.277698,47.463923,51.600858,0.506056,0.429002,0.475785,0.367049,0.377756,0.371256,0.703299,0.661550,0.686897,0.535174,0.519653,0.529076,0.189722,0.213532,0.199076,0.278147,0.241768,0.263855,65.781984,34.904190,53.651422,14.595461,10.523001,12.995566,10.990485,9.192399,10.284094,10.541393,8.888392,9.892000,35.757004,32.957098,34.657041
3,2003,1105,Alabama A&M,Southwest Athletic Conference,7,19.0,0.269231,1.069884,0.971322,0.997858,0.955355,0.926856,0.934529,102.402725,89.534708,92.999174,85.740188,106.597249,100.981886,16.662536,-17.062541,-7.982712,53.494178,48.566121,49.892905,0.488400,0.446777,0.457983,0.383144,0.317489,0.335166,0.667596,0.631796,0.641434,0.525370,0.474643,0.488300,0.232357,0.244086,0.240928,0.314651,0.233885,0.255629,64.127665,33.170400,41.505048,14.608038,13.405703,13.729408,5.012616,5.010599,5.011142,14.483337,11.279258,12.141895,36.651362,32.880417,33.895671
4,2003,1106,Alabama St,Southwest Athletic Conference,13,15.0,0.464286,1.075419,0.952266,1.009444,0.931644,0.932745,0.932234,99.498500,88.577978,93.648220,83.171774,104.243469,94.460182,16.326726,-15.665491,-0.811962,53.770968,47.613281,50.472207,0.517668,0.450522,0.481697,0.378952,0.323938,0.349480,0.714061,0.649253,0.679342,0.546506,0.486596,0.514411,0.258413,0.244313,0.250859,0.248100,0.157829,0.199740,74.138235,34.226174,52.756774,13.657301,11.636150,12.574542,9.413412,6.716493,7.968634,13.372223,11.337570,12.282230,30.130153,34.058097,32.234409


In [16]:
def tourn_round(DayNum):
    """
    Consolidate tournament rounds into meaningful info.
    """
    if (DayNum == 136) | (DayNum == 137):
        return 64
    elif (DayNum == 138) | (DayNum == 139):
        return 32
    elif (DayNum == 143) | (DayNum == 144):
        return 16
    elif (DayNum == 145) | (DayNum == 146):
        return 8
    elif DayNum == 152:
        return 4
    elif DayNum == 154:
        return 2
    else:
        return 68
    
df_tourney_all['tourn_round'] = df_tourney_all.DayNum.apply(tourn_round)

In [17]:
# Get the seed number by taking the last two characters of 'Seed' values:
df_seeds['seed'] = df_seeds['Seed'].apply(lambda x : int(x[1:3]))

In [18]:
# Drop the old 'Seed' column:
df_seeds = df_seeds[['Season', 'TeamID', 'seed']]

# Merge seeds, team names, and conference names with tournament data:
df_tourney_all = df_tourney_all.merge(df_seeds, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID']) \
.rename(columns={'seed': 'Wseed'}).drop(['TeamID'], axis=1) \
.merge(df_seeds, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID']) \
.rename(columns={'seed': 'Lseed'}).drop(['TeamID'], axis=1) \
.merge(win_teams, on='WTeamID').rename(columns={'TeamName': 'WTeamName'}) \
.merge(win_confs, on=['Season', 'WTeamID']).rename(columns={'Description': 'WConfName'}) \
.merge(lose_teams, on='LTeamID').rename(columns={'TeamName': 'LTeamName'}) \
.merge(lose_confs, on=['Season', 'LTeamID']).rename(columns={'Description': 'LConfName'})

In [19]:
# Calculate the point differential:
df_tourney_all['point_diff'] = df_tourney_all.WScore - df_tourney_all.LScore
df_tourney_all['upset'] = df_tourney_all.apply(lambda row: row.Wseed > row.Lseed, axis=1)
df_tourney_all['seed_dif'] = df_tourney_all.apply(lambda row: row.Wseed - row.Lseed, axis=1)
df_tourney_all = df_tourney_all[df_tourney_all['tourn_round'] != 68]
df_tourney_wins_total = df_tourney_all.groupby( [ "Season", "WTeamName"] ).size().to_frame(name = 'tourny_wins').reset_index()
df_tourney_all = df_tourney_all.merge(df_tourney_wins_total, how='left', left_on=['Season', 'WTeamName'], right_on=['Season', 'WTeamName'])
df_tourney_all.to_pickle('df_tourney_all_games_1985')

In [20]:
#Remove play-in games as those will mess up number of wins and don't tell us much of anything.
df_tourney_all_no_68 = df_tourney_all[df_tourney_all['tourn_round'] != 68]
#Add tourny wins to df_avgs
df_tourny_wins = df_tourney_all_no_68.groupby( [ "Season", "WTeamName"] ).size().to_frame(name = 'tourny_wins').reset_index()
df_avgs = df_avgs.merge(df_tourny_wins, how='left', left_on=['Season', 'TeamName'], right_on=['Season', 'WTeamName'], suffixes=('', '_y'))\
.drop('WTeamName', axis=1)
df_avgs = df_avgs.fillna(0)

In [21]:
df_team_spellings = pd.read_csv('mens-machine-learning-competition-2020/DataFiles/TeamSpellings.csv',encoding = "ISO-8859-1")
df_team_spellings = df_team_spellings.rename(columns={'TeamNameSpelling': 'TeamName'})
df_team_spellings.loc[len(df_team_spellings)] = ['texas-rgv', 1410]
df_team_spellings.loc[len(df_team_spellings)] = ['winston-salem state', 1445]
df_team_spellings.loc[len(df_team_spellings)] = ['st josephs', 1386]
df_team_spellings.loc[len(df_team_spellings)] = ['miss state', 1280]
df_team_spellings.loc[len(df_team_spellings)] = ['n carolina', 1314]
df_team_spellings.loc[len(df_team_spellings)] = ['ga tech', 1210]
df_team_spellings.loc[len(df_team_spellings)] = ['miami', 1274]
df_team_spellings.loc[len(df_team_spellings)] = ['boston col', 1130]
df_team_spellings.loc[len(df_team_spellings)] = ['wi-milwkee', 1454]
df_team_spellings.loc[len(df_team_spellings)] = ['w virginia', 1452]
df_team_spellings.loc[len(df_team_spellings)] = ['wi-grn bay', 1453]
df_team_spellings.loc[len(df_team_spellings)] = ['wash state', 1450]
df_team_spellings.loc[len(df_team_spellings)] = ['wm & mary', 1456]
df_team_spellings.loc[len(df_team_spellings)] = ['s carolina', 1376]
df_team_spellings.loc[len(df_team_spellings)] = ['youngs st', 1464]
df_team_spellings.loc[len(df_team_spellings)] = ['va military', 1440]
df_team_spellings.loc[len(df_team_spellings)] = ['la lafayette', 1418]
df_team_spellings.loc[len(df_team_spellings)] = ['ar lit rock', 1114]
df_team_spellings.loc[len(df_team_spellings)] = ['la monroe', 1419]
df_team_spellings.loc[len(df_team_spellings)] = ['tx-arlington', 1426]
df_team_spellings.loc[len(df_team_spellings)] = ['st johns', 1385]
df_team_spellings.loc[len(df_team_spellings)] = ['geo wshgtn', 1203]
df_team_spellings.loc[len(df_team_spellings)] = ['e tenn st', 1190]
df_team_spellings.loc[len(df_team_spellings)] = ['tx-arlington', 1426]
df_team_spellings.loc[len(df_team_spellings)] = ['col charlestn', 1158]
df_team_spellings.loc[len(df_team_spellings)] = ['nc-grnsboro', 1422]
df_team_spellings.loc[len(df_team_spellings)] = ['s florida', 1378]
df_team_spellings.loc[len(df_team_spellings)] = ['central fl', 1416]
df_team_spellings.loc[len(df_team_spellings)] = ['e carolina', 1187]
df_team_spellings.loc[len(df_team_spellings)] = ['va tech', 1439]
df_team_spellings.loc[len(df_team_spellings)] = ['u mass', 1269]
df_team_spellings.loc[len(df_team_spellings)] = ['geo mason', 1206]
df_team_spellings.loc[len(df_team_spellings)] = ['st bonavent', 1382]
df_team_spellings.loc[len(df_team_spellings)] = ['loyola-chi', 1260]
df_team_spellings.loc[len(df_team_spellings)] = ['n iowa', 1320]
df_team_spellings.loc[len(df_team_spellings)] = ['tx-san ant', 1427]
df_team_spellings.loc[len(df_team_spellings)] = ['s mississippi', 1379]
df_team_spellings.loc[len(df_team_spellings)] = ['tx el paso', 1431]
df_team_spellings.loc[len(df_team_spellings)] = ['middle tenn', 1292]
df_team_spellings.loc[len(df_team_spellings)] = ['st peters', 1389]
df_team_spellings.loc[len(df_team_spellings)] = ['u penn', 1335]
df_team_spellings.loc[len(df_team_spellings)] = ['ark pine bl', 1115]
df_team_spellings.loc[len(df_team_spellings)] = ['miss val st', 1290]
df_team_spellings.loc[len(df_team_spellings)] = ['alab a&m', 1105]
df_team_spellings.loc[len(df_team_spellings)] = ['grambling st', 1212]
df_team_spellings.loc[len(df_team_spellings)] = ['il-chicago', 1227]
df_team_spellings.loc[len(df_team_spellings)] = ['san fransco', 1362]
df_team_spellings.loc[len(df_team_spellings)] = ['st marys', 1388]
df_team_spellings.loc[len(df_team_spellings)] = ['loyola mymt', 1258]
df_team_spellings.loc[len(df_team_spellings)] = ['e washingtn', 1186]
df_team_spellings.loc[len(df_team_spellings)] = ['s utah', 1381]
df_team_spellings.loc[len(df_team_spellings)] = ['sac state', 1170]
df_team_spellings.loc[len(df_team_spellings)] = ['mt st marys', 1291]
df_team_spellings.loc[len(df_team_spellings)] = ['rob morris', 1352]
df_team_spellings.loc[len(df_team_spellings)] = ['nc-wilmgton', 1423]
df_team_spellings.loc[len(df_team_spellings)] = ['northeastrn', 1318]
df_team_spellings.loc[len(df_team_spellings)] = ['james mad', 1241]
df_team_spellings.loc[len(df_team_spellings)] = ['bham southern', 1128]
df_team_spellings.loc[len(df_team_spellings)] = ['tx christian', 1395]
df_team_spellings.loc[len(df_team_spellings)] = ['tx a&m-cc', 1394]
df_team_spellings.loc[len(df_team_spellings)] = ['ste f austin', 1372]
df_team_spellings.loc[len(df_team_spellings)] = ['sam hous st', 1358]
df_team_spellings.loc[len(df_team_spellings)] = ['nw state', 1322]
df_team_spellings.loc[len(df_team_spellings)] = ['s car state', 1354]
df_team_spellings.loc[len(df_team_spellings)] = ['beth-cook', 1126]
df_team_spellings.loc[len(df_team_spellings)] = ['maryland es', 1271]
df_team_spellings.loc[len(df_team_spellings)] = ['s alabama', 1375]
df_team_spellings.loc[len(df_team_spellings)] = ['app state', 1111]
df_team_spellings.loc[len(df_team_spellings)] = ['n mex state', 1308]
df_team_spellings.loc[len(df_team_spellings)] = ['cal st nrdge', 1169]
df_team_spellings.loc[len(df_team_spellings)] = ['lg beach st', 1253]
df_team_spellings.loc[len(df_team_spellings)] = ['tx-pan am', 1410]
df_team_spellings.loc[len(df_team_spellings)] = ['bowling grn', 1132]
df_team_spellings.loc[len(df_team_spellings)] = ['central mich', 1141]
df_team_spellings.loc[len(df_team_spellings)] = ['n arizona', 1319]
df_team_spellings.loc[len(df_team_spellings)] = ['tn tech', 1399]
df_team_spellings.loc[len(df_team_spellings)] = ['tn state', 1398]
df_team_spellings.loc[len(df_team_spellings)] = ['se missouri', 1369]
df_team_spellings.loc[len(df_team_spellings)] = ['jksnville st', 1240]
df_team_spellings.loc[len(df_team_spellings)] = ['sacred hrt', 1357]
df_team_spellings.loc[len(df_team_spellings)] = ['n hampshire', 1306]
df_team_spellings.loc[len(df_team_spellings)] = ['nc-asheville', 1421]
df_team_spellings.loc[len(df_team_spellings)] = ['gard-webb', 1205]
df_team_spellings.loc[len(df_team_spellings)] = ['charl south', 1149]
df_team_spellings.loc[len(df_team_spellings)] = ['maryland bc', 1420]
df_team_spellings.loc[len(df_team_spellings)] = ['fla atlantic', 1194]
df_team_spellings.loc[len(df_team_spellings)] = ['la tech', 1256]
df_team_spellings.loc[len(df_team_spellings)] = ['s methodist', 1374]
df_team_spellings.loc[len(df_team_spellings)] = ['st fran (pa)', 1384]
df_team_spellings.loc[len(df_team_spellings)] = ['st fran (ny)', 1383]
df_team_spellings.loc[len(df_team_spellings)] = ['utah val st', 1430]
df_team_spellings.loc[len(df_team_spellings)] = ['cs bakersfld', 1167]
df_team_spellings.loc[len(df_team_spellings)] = ['grd canyon', 1213]
df_team_spellings.loc[len(df_team_spellings)] = ['neb omaha', 1303]
df_team_spellings.loc[len(df_team_spellings)] = ['abl christian', 1101]
df_team_spellings.loc[len(df_team_spellings)] = ['incar word', 1230]
df_team_spellings.loc[len(df_team_spellings)] = ['n florida', 1316]
df_team_spellings.loc[len(df_team_spellings)] = ['fla gulf cst', 1195]
df_team_spellings.loc[len(df_team_spellings)] = ['mass lowell', 1262]
df_team_spellings.loc[len(df_team_spellings)] = ['central ark', 1146]
df_team_spellings.loc[len(df_team_spellings)] = ['siu edward', 1188]
df_team_spellings.loc[len(df_team_spellings)] = ['kennesaw st', 1244]
df_team_spellings.loc[len(df_team_spellings)] = ['wins-salem', 1445]

In [22]:
df_rpi = pd.read_pickle('rpi')
df_rpi['TeamName'] = df_rpi['TeamName'].str.lower()

df_new = df_rpi.merge(df_team_spellings, on=['TeamName'])
df_new = df_new.astype({"Season": int, "TeamID": int})
df_avgs = df_avgs.merge(df_new, on=['Season', 'TeamID'], suffixes=('', '_y'))
df_avgs.drop(list(df_avgs.filter(regex='_y$')), axis=1, inplace=True)

In [23]:
df_kenpom = pd.read_pickle('kenpom_df')
df_kenpom['TeamName'] = df_kenpom['TeamName'].str.lower()

df_new = df_kenpom.merge(df_team_spellings, on=['TeamName'])
df_new = df_new.astype({"Season": int, "TeamID": int})
df_avgs = df_avgs.merge(df_new, on=['Season', 'TeamID'], suffixes=('', '_y'))
df_avgs.drop(list(df_avgs.filter(regex='_y$')), axis=1, inplace=True)

In [24]:
#Add seed to avgs, fill 0 for teams that do not make tourny
df_avgs = df_avgs.merge(df_seeds, how='left', on=['TeamID', 'Season'])
df_avgs['seed'].fillna(0, inplace=True)

# Remove teams with blank rpis. 16 teams from 2004 including 3 teams that made the tourny one of which is 
# 2004 champs uconn.
blank_rpi_index = df_avgs[df_avgs['rpi'] == '--'].index
df_avgs.drop(blank_rpi_index, inplace=True)

In [25]:
def convert_to_int(row):
    if(row['kp_rating'][0] == '+'):
        value = float(row['kp_rating'][1:])
    else:
        value = -float(row['kp_rating'][1:])
    row['kp_rating'] = value
    return row
    
df_avgs = df_avgs.apply(convert_to_int, axis=1)

In [26]:
df_avgs['made_tourn'] = df_avgs.apply(lambda row: row.seed > 0, axis=1)

In [27]:
df_avgs.to_pickle('df_avgs_w_l')
df_team_spellings.to_pickle('df_spellings')
# df_tourny_wins.to_pickle('df_tourny_wins')